# Analyse der heisseste und kälteste Messstationen

Dieses Notebook kann lokal oder **direkt im Browser** auf [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/meteotest/urban-heat-API-docs/data-analysis?labpath=python_data_analysis_hottest_coldest_locations.ipynb) oder [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meteotest/urban-heat-API-docs/blob/data-analysis/python_data_analysis_hottest_coldest_locations.ipynb) ausgeführt werden.

In [16]:
import requests 
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt # for plotting
import folium
import branca
import math
from typing import Tuple

## Berechnungen heisseste und kälteste Messstation

In [17]:
# Auswahl an Zeitraum (z.B. eine Hitzewelle, ein Monat, oder der ganze Sommer)
# Daten (UTC) als String in folgendem Format: "YYYY-MM-DDThh:mm:ssZ"
time_from = "2024-08-01T00:00:00Z"
time_to = "2024-08-02T23:59:59Z"

# Auswahl an Stationen
# Wenn alle Stationen berücksichtigt werden sollen, dann einfach die Liste leer lassen
# Hier ist eine Liste aller Stationen mit Name und ID: https://smart-urban-heat-map.ch/api/v2/stations
# station_ids = ["11001", "11002"] # Beispiel für nur eine Auswahl von zwei Stationen
station_ids = [] # Beispiel für alle Stationen

In [28]:
def get_stations() -> gpd.GeoDataFrame:
    response = requests.get(
            url=f"https://smart-urban-heat-map.ch/api/v2/stations"
    )
    stations = gpd.read_file(response.text)
    stations.drop(["latestMeasurementDate"], axis=1, inplace=True)

    return stations

def get_station_analysis_mean_temp(time_from: str, time_to: str, station_ids: [str]) -> pd.DataFrame:
    stations = get_stations()
    if station_ids:
        stations = stations[stations["stationId"].isin(station_ids)]
    
    time_from_dt = pd.to_datetime(time_from)
    time_to_dt = pd.to_datetime(time_to)
    time_difference = time_to_dt - time_from_dt
    min_expected_values = round(time_difference.days * 24 * 2) # minimum two values per hour
        
    stations["mean_temperature"] = None
    stations["max_temperature"] = None
    stations["date_of_max_temperature"] = None
    stations["min_temperature"] = None
    stations["date_of_min_temperature"] = None

    
    for idx, station in stations.iterrows():
        station_id = station.stationId
        response = requests.get(url=f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id}&timeFrom={time_from}&timeTo={time_to}")
        
        payload = response.json()
        if payload["values"] is None or not len(payload["values"]): 
            stations = stations.drop(idx)
            continue
            
        if len(payload["values"]) < min_expected_values:
            stations = stations.drop(idx)
            continue
                        
        df = pd.DataFrame(payload["values"])

        df["dateObserved"] = pd.to_datetime(df["dateObserved"])
        df["dateObserved"] = df["dateObserved"].dt.tz_convert("Europe/Zurich")
        
        # hier wird der mittelwert berechnet
        mean_temperature = mean_temperatures(df)
        
        # hier werden die min und max temperaturen berechnet
        # und das jeweilige datum herausgeschrieben
        min_temperature = min_temperatures(df)[0]
        date_of_min_temperature = min_temperatures(df)[1]
        max_temperature = max_temperatures(df)[0]
        date_of_max_temperature = max_temperatures(df)[1]

        # hier wird der mittelwert dem 'stations' dataframe hinzugefügt
        stations.at[idx, "mean_temperature"] = mean_temperature
        
        # hier werden der min und max temperatur sowie deren messdatum dem 'stations' dataframe hinzugefügt
        stations.at[idx, "min_temperature"] = min_temperature
        stations.at[idx, "date_of_min_temperature"] = date_of_min_temperature
        stations.at[idx, "max_temperature"] = max_temperature
        stations.at[idx, "date_of_max_temperature"] = date_of_max_temperature

    return stations

    
def mean_temperatures(df: pd.DataFrame) -> float:
    mean_temp = df.mean()["temperature"]
    return mean_temp

def min_temperatures(df: pd.DataFrame) -> tuple[float,str]:
    min_temp = df.min()["temperature"]
    dateof_min_temp = df.loc[df['temperature'].idxmin()]['dateObserved']
    return min_temp, dateof_min_temp

def max_temperatures(df: pd.DataFrame) -> tuple[float,str]:
    max_temp = df.max()["temperature"]
    dateof_max_temp = df.loc[df['temperature'].idxmax()]['dateObserved']
    return max_temp, dateof_max_temp

In [29]:
# run analysis of mean temperatures
# this can take a few seconds or minutes depending on the chosen time interval
station_analysis = get_station_analysis_mean_temp(time_from, time_to, station_ids)
station_analysis

,name,stationId,geometry,mean_temperature,max_temperature,date_of_max_temperature,min_temperature,date_of_min_temperature
0,"Ausserholligen 2, ewb",11001,POINT (7.40642 46.94542),19.469848,21.886396,2024-08-01 11:27:09+02:00,18.521782,2024-08-01 06:07:09+02:00
1,Bundesplatz,11002,POINT (7.44353 46.94692),20.160474,23.603418,2024-08-01 11:29:33+02:00,18.775463,2024-08-01 09:49:33+02:00
2,"Breitenrain, Waffenweg",11003,POINT (7.45192 46.96173),19.665721,22.695507,2024-08-01 11:43:35+02:00,18.66598,2024-08-01 09:23:35+02:00
3,Schosshaldenfriedhof 2,11004,POINT (7.47186 46.95339),19.018205,24.364462,2024-08-01 11:55:43+02:00,17.229343,2024-08-01 05:05:43+02:00
4,Monbijou-Park,11005,POINT (7.43462 46.94187),19.743076,24.871824,2024-08-01 11:51:36+02:00,18.516441,2024-08-01 05:11:36+02:00
...,...,...,...,...,...,...,...,...
135,Spielplatz Stiglimatt,12007,POINT (7.2992 47.07154),20.825704,26.062792,2024-08-01 11:44:23+02:00,17.953003,2024-08-01 05:24:22+02:00
136,Sportanlage Grien,12008,POINT (7.29574 47.07508),20.550755,25.753033,2024-08-01 11:45:32+02:00,17.710003,2024-08-01 05:35:31+02:00
137,Bahnhofplatz Busswil,12009,POINT (7.31932 47.0987),21.19173,27.307163,2024-08-01 11:52:35+02:00,18.233387,2024-08-01 05:12:35+02:00
138,Reitplatz Grünau,12010,POINT (7.30167 47.07714),20.610676,27.077517,2024-08-01 11:38:33+02:00,17.496376,2024-08-01 05:28:33+02:00


## Liste der mittleren Temperaturen

### Wärmste *n* Standorte herauslesen

Hier muss für *n_warmest* eine Zahl eingegeben werden.

In [93]:
# select n coldest stations
n_warmest = 3 # f.ex. put a value of 3 for top 3 coldest stations

In [94]:
# Sort in ascending order to get hottest stations
df_warm = station_analysis.sort_values(by='mean_temperature', ascending=False)[:n_warmest]
df_warm

,name,stationId,geometry,mean_temperature
130,Schulhaus Grentschel,12002,POINT (7.31128 47.07855),21.63653
133,Marktplatz Lyss,12005,POINT (7.30584 47.0735),21.6016
129,Mühleplatz Lyss,12001,POINT (7.30895 47.0709),21.429237


### Kühlste *n* Standorte

Hier muss für *c_coldest* eine Zahl eingegeben werden.

In [95]:
# select n coldest stations
n_coldest = 3 # f.ex. put a value of 3 for top 3 coldest stations

In [96]:
# Sort in ascending order to get coldest stations
df_cold = station_analysis.sort_values(by='mean_temperature', ascending=True)[:n_coldest]
df_cold

,name,stationId,geometry,mean_temperature
47,Bremgartenwald,11050,POINT (7.42128 46.96512),17.359188
92,Köniz Bläuacker Kandelaber auf Platz,11095,POINT (7.41466 46.92327),17.669947
106,Ostermundigen BäreTower,11109,POINT (7.48114 46.95563),17.773942


## Karte der mittleren Temperaturen pro Messstation

In [86]:
# mean_temperature Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

# Add a fixed title to the map
title_html = f'''
     <div style="position: fixed; 
     top: 20px; left: 100px; width: 25%; height: 45px; 
     background-color: #F0F0F0; border: 1px solid black; z-index: 9999; font-size: 14px; font-weight: bold;">
     Mittlere Temperatur <br> vom {pd.to_datetime(time_from).strftime('%d.%m.%Y')} bis {pd.to_datetime(time_to).strftime('%d.%m.%Y')}
     </div>
     '''
m.get_root().html.add_child(folium.Element(title_html))

colormap = branca.colormap.linear.YlOrRd_09
mean_temperature = station_analysis.mean_temperature.values

# Define colourmap range depending on values of 'mean_temperature
vmin = math.floor(mean_temperature.min())
vmax = math.ceil(mean_temperature.max())

# Define the colormap with the specified range
colormap = branca.colormap.linear.YlOrRd_09.scale(vmin, vmax)

# Convert to step colormap with a specified number of steps
n_steps = int((vmax - vmin) / 0.5)  # Define the number of steps
colormap = colormap.to_step(n_steps)

# colormap = colormap.scale(0, mean_temperature).to_step(mean_temperature) 
colormap.caption = "Mittlere Temperatur"
colormap.add_to(m)

# plot each station temperature
for idx, station in station_analysis.iterrows():
    color = colormap(station.mean_temperature)
    # text with temperature value
    folium.Marker(
        location=(station.geometry.y, station.geometry.x),
        icon=folium.DivIcon(
            html=f'<div style="font-size: 10pt; color: {color}; text-shadow: -1px -1px 0 #D3D3D3, 1px -1px 0 #D3D3D3, -1px 1px 0 #D3D3D3, 1px 1px 0 #D3D3D3;">{station.mean_temperature:.1f}°C</div>'
            ),
        tooltip=f"{station['name']}: Mittlere Temperatur {station.mean_temperature:.1f} °C",
    ).add_to(m)

# show map
m